In [1]:
import pandas as pd
import re
import requests
import numpy as np
from bs4 import BeautifulSoup
import re
from pathlib import Path

import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [2]:
def get_page_count():
    base_url = "https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/"
    page = requests.get(base_url)
    soup = BeautifulSoup(page.content, "html.parser")
    nav = soup.find("nav", class_="pages")
    a_fetch = nav.find_all("a", class_="step")
    last_ele = a_fetch[-1].text
    return int(last_ele)

In [3]:
page_count = get_page_count()
pub_urls = []
for i in range(page_count):
    url = "https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=" + f"{i}"
    pub_urls.append(url)
    
pub_urls

['https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=0',
 'https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=1',
 'https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=2',
 'https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=3',
 'https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=4',
 'https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=5',
 'https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=6',
 'https://pureportal.coventry.ac.uk/en/organisations/school-of-economics-finance-and-accounting/publications/?page=7',
 'https://pureportal.coventry.ac.uk/en/organisat

In [4]:
pub_list = []

for url in pub_urls:
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    for h3 in soup.find_all("h3", class_="title"):
        a_fetch = h3.find("a")
        link = a_fetch.attrs["href"]
        title = h3.findChild('span').text
        pub_map = {}
        pub_map["pub_link"] = link
        pub_map["pub_title"] = title
        pub_list.append(pub_map)

In [5]:
pub_list

[{'pub_link': 'https://pureportal.coventry.ac.uk/en/publications/a-bibliometric-review-of-the-waqf-literature',
  'pub_title': 'A bibliometric review of the Waqf literature'},
 {'pub_link': 'https://pureportal.coventry.ac.uk/en/publications/a-note-on-covid-19-instigated-maximum-drawdown-in-islamic-markets',
  'pub_title': 'A note on COVID-19 instigated maximum drawdown in Islamic markets versus conventional counterparts'},
 {'pub_link': 'https://pureportal.coventry.ac.uk/en/publications/bank-stock-valuation-theories-do-they-explain-prices-based-on-the',
  'pub_title': 'Bank stock valuation theories: do they explain prices based on theories?'},
 {'pub_link': 'https://pureportal.coventry.ac.uk/en/publications/ceo-duality-and-firm-performance-a-systematic-review-and-research',
  'pub_title': 'CEO Duality and Firm Performance: A Systematic Review and Research Agenda'},
 {'pub_link': 'https://pureportal.coventry.ac.uk/en/publications/ceo-financial-experience-and-firms-earnings-management-in

In [6]:
def get_pub_date(soup):
    for div in soup.find_all("tr", class_="status"):
        a_fetch = div.find("span", class_="date").text
        
    return a_fetch

In [7]:
def get_auth(soup):
    auth_name = []
    auth_link = []
    for div in soup.find_all("a", class_="link person"):
        link = div.attrs["href"]
        name = div.find("span").text
        auth_name.append(name)
        auth_link.append(link)

    return (auth_name,auth_link)

In [8]:
for index, pub in enumerate(pub_list):
    url = pub["pub_link"]
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser")
    at_name,link = get_auth(soup)
    date = get_pub_date(soup)
    pub_list[index]["pub_date"] = date
    pub_list[index]["auth_name"] = at_name
    pub_list[index]["auth_link"] = link

In [9]:
pub = pub_list[0]
url = pub["pub_link"]
page = requests.get(url)
soup = BeautifulSoup(page.content, "html.parser")
name,link = get_auth(soup)
name,link

(['Rashedul Hasan'],
 ['https://pureportal.coventry.ac.uk/en/persons/rashedul-hasan'])

In [31]:
df = pd.DataFrame(pub_list)

In [32]:
df['id'] = df.index

In [33]:
df.head()

,pub_link,pub_title,pub_date,auth_name,auth_link,id
0,https://pureportal.coventry.ac.uk/en/publicati...,A bibliometric review of the Waqf literature,Jun 2022,[Rashedul Hasan],[https://pureportal.coventry.ac.uk/en/persons/...,0
1,https://pureportal.coventry.ac.uk/en/publicati...,A note on COVID-19 instigated maximum drawdown...,May 2022,[Rashedul Hasan],[https://pureportal.coventry.ac.uk/en/persons/...,1
2,https://pureportal.coventry.ac.uk/en/publicati...,Bank stock valuation theories: do they explain...,1 Mar 2022,[Alireza Zarei],[https://pureportal.coventry.ac.uk/en/persons/...,2
3,https://pureportal.coventry.ac.uk/en/publicati...,CEO Duality and Firm Performance: A Systematic...,25 May 2022,[Mei Yu],[https://pureportal.coventry.ac.uk/en/persons/...,3
4,https://pureportal.coventry.ac.uk/en/publicati...,CEO Financial Experience and Firms’ Earnings M...,7 Mar 2022,"[Thai Nguyen, Thang Nguyen, Panagiotis Andriko...",[https://pureportal.coventry.ac.uk/en/persons/...,4


In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 637 entries, 0 to 636
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   pub_link   637 non-null    object
 1   pub_title  637 non-null    object
 2   pub_date   637 non-null    object
 3   auth_name  637 non-null    object
 4   auth_link  637 non-null    object
 5   id         637 non-null    int64 
dtypes: int64(1), object(5)
memory usage: 30.0+ KB


In [14]:
df[df['id']==4]["pub_title"]

4    CEO Financial Experience and Firms’ Earnings M...
Name: pub_title, dtype: object

In [35]:
stemmer = PorterStemmer()

def rem_punc_low_case(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

def clean_data(text):
    text = rem_punc_low_case(text)
    text = text.split()
    text = [stemmer.stem(word) for word in text if not word in stopwords.words('english')]
    text = ' '.join(text)
    return text

In [36]:
def clean_title(title_list):
    title = ' '.join(title_list)
    title = re.sub('[^a-zA-Z]', ' ', title)
    title = title.lower()
    return title

In [37]:
df['text'] = df['pub_title'].apply(lambda x: clean_data(x))

In [38]:
df['auth_name_extract'] = df['auth_name'].apply(lambda x: clean_title(x))

In [39]:
df['pub_date'] = df['pub_date'].apply(lambda x: rem_punc_low_case(x))

In [87]:
df[df['id']==22]["auth_name_extract"]

22    jaliyyah bello mohammad khaleq newaz
Name: auth_name_extract, dtype: object

In [86]:
df["text"] = df["text"] + " " + df["auth_name_extract"] + " " + df["pub_date"]

In [56]:
crawl_df = new = df.filter(['text'], axis=1).copy()

In [57]:
crawl_df.head()

,text
0,bibliometr review waqf literatur rashedul hasa...
1,note covid19 instig maximum drawdown islam mar...
2,bank stock valuat theori explain price base th...
3,ceo dualiti firm perform systemat review resea...
4,ceo financi experi firm earn manag import expe...


In [66]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
# fits and transform -> vector
X = vectorizer.fit_transform(crawl_df["text"])
# To transpose matrix
X = X.T.toarray()
# Indexing the vocab and set in df
vect_df = pd.DataFrame(X, index=vectorizer.get_feature_names_out())

In [67]:
vect_df.head()

,0,1,2,3,4,5,6,7,8,9,...,627,628,629,630,631,632,633,634,635,636
10,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.276464,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
13,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [142]:
def get_similar_articles(q, sim_df):
  print("search query =", q)
  q = [q]
  q = [stemmer.stem(word) for word in q if not word in stopwords.words('english')]
  q_vec = vectorizer.transform(q).toarray().reshape(sim_df.shape[0],)
  sim = {}
  # Calc similarity
  for j in range(10):
    sim[j] = np.dot(sim_df.loc[:, j].values, q_vec) / np.linalg.norm(sim_df.loc[:, j]) * np.linalg.norm(q_vec)
  
  sim_sorted = sorted(sim.items(), key=lambda x: x[1], reverse=True)
  # list publications & its similarity
  has_val = False
  for k, v in sim_sorted:
    if v != 0.0:
      has_val = True
      print("Cosine Similarities:", v)
#       print(crawl_df["text"][k])
      print(df["pub_title"][k] + " " + df["auth_name_extract"][k] + " " + df["pub_date"][k])
      print()
  if has_val == False:
    print("No results found")

In [2]:
# Add The Query
while True:
    q1 = input()
# Call the function
    get_similar_articles(q1.lower(), vect_df)

d


NameError: name 'get_similar_articles' is not defined